In [1]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from scipy import ndimage
from sklearn.model_selection import train_test_split

2024-06-17 13:45:41.274216: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 13:45:42.380308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 13:45:42.380371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 13:45:42.551480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 13:45:42.803397: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
sub_window_and_count_path_train = "../../S_lab_TasselNet/Method_2/final_train_sub_windows_and_counts"
sub_window_and_count_path_valid = "../../S_lab_TasselNet/Method_2/final_valid_sub_windows_and_counts"

In [4]:
%%time
# load the data

# subwindow data
X_train = np.load(os.path.join(sub_window_and_count_path_train, 'train_sub_windows.npy'))
print(X_train.shape)

# sub counts data
y_train = np.load(os.path.join(sub_window_and_count_path_train,'train_sub_counts.npy'))
print(y_train.shape)

(1572864, 32, 32, 3)
(1572864,)
CPU times: user 110 ms, sys: 29.9 s, total: 30 s
Wall time: 1min 7s


In [5]:
%%time
# load validation data
# sub windows
X_valid = np.load(os.path.join(sub_window_and_count_path_valid,'valid_sub_windows.npy'))
print(X_valid.shape)

# sub counts data
y_valid = np.load(os.path.join(sub_window_and_count_path_valid,'valid_sub_counts.npy'))
print(y_valid.shape)

(393216, 32, 32, 3)
(393216,)
CPU times: user 5.97 ms, sys: 6.79 s, total: 6.79 s
Wall time: 15 s


In [6]:
# load and add the new layers to the prev trained model
gmp_model = tf.keras.models.load_model('../../S_lab_TasselNet/models/tasselnet_global_max_pool_04_23_2022.h5')
# look at the input shape
gmp_model.input

2024-06-17 13:47:38.529803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>

In [7]:
gmp_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [8]:
# need to remove some of the last layers
output_trial = gmp_model.layers[-7].output

In [9]:
# create the functional API model
reduced_model = tf.keras.models.Model(gmp_model.input, output_trial)
reduced_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [10]:
# okay, now need to add back the dropout, the dense and activation

# add dropout
added_dropout = tf.keras.layers.Dropout(0.5, name = "New_dropout")(gmp_model.layers[-7].output)

# add global max pooling
added_flatten = tf.keras.layers.GlobalMaxPooling2D()(added_dropout)

# add dense
added_dense = tf.keras.layers.Dense(512, name = "New_Dense")(added_flatten)

# add activation
added_Act = tf.keras.layers.Activation('relu', name = "New_Activation")(added_dense)

# add dropout
added_dropout2 = tf.keras.layers.Dropout(0.5, name = "New_dropout2")(added_Act)

# add dense
added_dense2 = tf.keras.layers.Dense(1, name = "New_Dense2")(added_dropout2)

# add activation
added_Act2 = tf.keras.layers.Activation('relu', name = "New_Activation2")(added_dense2)

# define the new model with functional API
new_model = tf.keras.models.Model(gmp_model.input, added_Act2)

In [11]:
# print the model summary
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

This is where we need to write the function - Rylie has two functions a generator, and also a batch-wise prediction function. May be we only need the generator function? Let's see.

In [12]:
def data_generator(x_data, y_data, batch_size, shuffle=False, peek=False, verbose=False):
    num_samples = len(x_data)
    indices = np.arange(num_samples)
    
    if peek:    # Give first batch unshuffled and don't change start index when peeking for training
        end = min(batch_size, num_samples)
        if verbose:
            print(f"Generating peeking batch up to index {end}")
        batch_x = x_data[:end]
        batch_y = y_data[:end]
        peek = False
        yield (batch_x, batch_y)

    while True:    # Loop indefinitely for epochs
        # Shuffle indices at the start of each epoch after the peek, if shuffle is enabled
        if shuffle:
            np.random.shuffle(indices)
        
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]

            # Print batch indices if verbose
            if verbose:
                # Warning: calling verbose when shuffling will usually clutter output
                if shuffle:
                    if len(batch_indices) < 16:
                        print(f"Batch indices: {np.sort(batch_indices)}")
                    else:
                        print(f"Printing batch indices would clutter output. Skipped.")
                    print(f"Length: {len(batch_indices)}")
                else:
                    print(f"Generating batch from index {start} to {end}")
                # traceback.print_stack()

            # Generate batches
            batch_x = x_data[batch_indices]
            batch_y = y_data[batch_indices]

            # Yield the current batch
            yield (batch_x, batch_y)

In [13]:
%%time
batch_size = 1024
generator_batch_size = 1024
train_gen = data_generator(X_train, y_train, batch_size, shuffle=False, peek=True, verbose=False)
val_gen = data_generator(X_valid, y_valid, generator_batch_size, shuffle=False, peek=True, verbose=False)

CPU times: user 2 µs, sys: 6 µs, total: 8 µs
Wall time: 11.9 µs


In [14]:
steps_per_epoch = int(np.ceil(len(X_train) / batch_size))
train_steps = int(np.ceil(len(X_train) / generator_batch_size))
validation_steps = int(np.ceil(len(X_valid) / generator_batch_size))

In [15]:
steps_per_epoch, train_steps, validation_steps

(1536, 1536, 384)

In [16]:
%%time
# frozen model

#     freeze layers
for layer in gmp_model.layers:
    layer.trainable = False
    
# compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
# add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=10)

# fit the generator - the steps are a must here
history = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])

Epoch 1/100


2024-06-17 13:47:40.869803: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-06-17 13:47:41.497794: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-06-17 13:47:48.031817: I external/local_xla/xla/service/service.cc:168] XLA service 0x14fbaed56400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-17 13:47:48.031862: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2024-06-17 13:47:48.091184: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1718650068.584183 2002094 device_compiler.h:186] Compiled cluster using XLA!  T

1536/1536 [==============================] - 35s 17ms/step - loss: 0.0279 - mean_absolute_error: 0.0588 - val_loss: 0.0218 - val_mean_absolute_error: 0.0329
Epoch 2/100
1536/1536 [==============================] - 27s 17ms/step - loss: 0.0257 - mean_absolute_error: 0.0590 - val_loss: 0.0212 - val_mean_absolute_error: 0.0340
Epoch 3/100
1536/1536 [==============================] - 28s 18ms/step - loss: 0.0253 - mean_absolute_error: 0.0596 - val_loss: 0.0208 - val_mean_absolute_error: 0.0349
Epoch 4/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0252 - mean_absolute_error: 0.0600 - val_loss: 0.0207 - val_mean_absolute_error: 0.0354
Epoch 5/100
1536/1536 [==============================] - 27s 17ms/step - loss: 0.0252 - mean_absolute_error: 0.0601 - val_loss: 0.0205 - val_mean_absolute_error: 0.0362
Epoch 6/100
1536/1536 [==============================] - 26s 17ms/step - loss: 0.0251 - mean_absolute_error: 0.0600 - val_loss: 0.0203 - val_mean_absolute_error: 0.037

In [17]:
%%time
# # unfreeze few layers and retrain - finetune the model
gmp_model.trainable = True

set_trainable = False

for layer in gmp_model.layers:
    if layer.name == 'conv2d_3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
            
    # # compile the mdoel
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
    # # add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=15)
    
    # # fit the model (fine tuned)
history_new = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])


Epoch 1/100


2024-06-17 14:02:09.240593: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1536/1536 [==============================] - 31s 18ms/step - loss: 0.0308 - mean_absolute_error: 0.0500 - val_loss: 0.0225 - val_mean_absolute_error: 0.0327
Epoch 2/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0282 - mean_absolute_error: 0.0541 - val_loss: 0.0246 - val_mean_absolute_error: 0.0988
Epoch 3/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0338 - mean_absolute_error: 0.0481 - val_loss: 0.0225 - val_mean_absolute_error: 0.0327
Epoch 4/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0337 - mean_absolute_error: 0.0480 - val_loss: 0.0225 - val_mean_absolute_error: 0.0327
Epoch 5/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0337 - mean_absolute_error: 0.0480 - val_loss: 0.0225 - val_mean_absolute_error: 0.0327
Epoch 6/100
1536/1536 [==============================] - 27s 18ms/step - loss: 0.0337 - mean_absolute_error: 0.0480 - val_loss: 0.0225 - val_mean_absolute_error: 0.032

In [18]:
# save the finetuned model
model_name = 'trained_gmp_model.h5'
new_model.save('models' + '/' + model_name)

/common/statsgeneral/gayara/tfp_for_TN/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
